## 5 如何分析SEO的排名要素？請從量化與質化數據個別探討

### 5.1 爬取google第一到第三頁關於心理諮商的標題/meta/內容

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# 设置Selenium浏览器
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

def scrape_google_search(query, pages=3):
    titles = []
    meta_descriptions = []
    contents = []
    
    for page in range(pages):
        driver.get(f"https://www.google.com/search?q={query}&start={page * 10}")
        time.sleep(3)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_results = soup.select("div.g")
        
        for result in search_results:
            try:
                title_element = result.select_one("h3")
                if title_element:
                    title = title_element.get_text()
                    link = result.select_one("a")['href']
                    
                    titles.append(title)
                    
                    # 打开新标签页并抓取内容
                    driver.execute_script("window.open(arguments[0], '_blank');", link)
                    driver.switch_to.window(driver.window_handles[1])
                    time.sleep(3)
                    
                    inner_soup = BeautifulSoup(driver.page_source, 'html.parser')
                    content = inner_soup.get_text()
                    meta_description_element = inner_soup.select_one("meta[name='description']")
                    meta_description = meta_description_element['content'] if meta_description_element else "N/A"
                    
                    contents.append(content)
                    meta_descriptions.append(meta_description)
                    
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
            except Exception as e:
                print(f"Error processing result: {e}")
                continue
    
    return titles, meta_descriptions, contents

query = "心理諮商"
titles, meta_descriptions, contents = scrape_google_search(query, pages=3)

# 打印抓取的标题、meta描述和内容
print("Titles:", titles)
print("Meta Descriptions:", meta_descriptions)
print("Contents:", contents)

# 关闭浏览器
driver.quit()


Titles: ['也許該找人聊聊-何謂心理諮商服務？', '心理諮商是什麼？分享諮商項目、費用與心理治療所推薦 ...', '心理諮商有用嗎？心理諮商流程、費用、免費資源懶人包', '心理諮商費用怎麼算？有免費心理諮商嗎？諮商收費標準 - 親子天下', '也許該找人聊聊-何謂心理諮商服務？', '心理諮商有用嗎？心理諮商流程、費用、免費資源懶人包', '有我傾聽」 「年輕族群心理健康支持方案」8月1日上路！', '有我傾聽」 「年輕族群心理健康支持方案」8月1日上路！', '心理諮商/心理治療是什麼？貴嗎？心理師：有效治療有這十二 ...', '各縣市心理諮商合作機構名單- 心理健康司', '認識諮商心理師', '台灣心理諮商資訊網', '心理諮商費用？免費、自費差異？心理諮商項目、重點介紹', '心理諮商有效嗎?心理師：先讀這五個重點！', '臺灣諮商心理學會', '心理諮商', '治療師陣容', '心理諮商/治療常見問答FAQ', '心理諮商/治療是什麼？有用嗎？心理師帶你完整認識心理諮商', '諮商心理學', '心理諮商服務(採預約制)', '重要公告】衛生福利部112-113年度15-30歲年輕族群心理 ...', '心理諮商預約', '台北市合格心理諮商機構', '綻芯心理諮商所', '華人心理治療基金會', '心理諮商預約----彰化縣社區心理衛生中心', '心理諮商服務(採預約制)', '聯合心理諮商所', '民眾心理諮商服務', '諮商心理學- 維基百科，自由的百科全書', '心理諮商–精神醫學、自律神經專家、失眠、焦慮']
Meta Descriptions: ['N/A', '心理諮商有用嗎？與心理治療差在哪？本文將推薦你台北的知芯心理治療所，並介紹心理諮詢的進行方式、談論議題、健保給付，以及自費心理諮商所和心理治療所的服務差異、挑選要點與費用，陪伴你度過人生低谷！', '心理諮商有用嗎？心理諮商是什麼？是許多民眾對「心理諮商」常有的疑惑。台灣心理諮商所數量6年內成長近2倍、愈來愈多人遇到內在困擾時會尋求諮商協助，讓台灣進入全民諮商時代。心理諮商流程、費用如何？什麼程度...', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '臺灣諮商心

In [ ]:
# to data frame

### 5.2  用CKIP進行斷詞斷句並輸出到googlesheet(串接pygsheet套件)

#### 斷詞斷句

In [28]:

from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import pygsheets

# data_utils.download_data_url("./ckip_model")
# data_utils.download_data_gdown("./") # gdrive-ckip


# 載入CKIP模型
ws = WS("./data")

# 斷詞
word_sentences_titles = ws(titles)
word_sentences_meta = ws(meta_descriptions)
word_sentences_contents = ws(contents)


#### 輸出到Google Sheets

In [30]:
gc = pygsheets.authorize(service_file='my-project-240608-425814-fb66d30e0809.json')
sh = gc.create('SEO分析結果')
wks = sh.add_worksheet('分詞結果')
wks.update_values('A1', [['類別', '原文', '分詞結果']])
wks.update_values('A2', [['標題', orig, ' '.join(words)] for orig, words in zip(titles, word_sentences_titles)])
wks.update_values(f'A{2+len(titles)}', [['Meta描述', orig, ' '.join(words)] for orig, words in zip(meta_descriptions, word_sentences_meta)])


def split_long_text(text, max_length=50000):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

row = 2

# 因為內容超過 Google Sheets 單一儲存格的字數上限
for orig, words in zip(contents, word_sentences_contents):
    segments = split_long_text(' '.join(words))
    for segment in segments:
        wks.update_values(f'A{row}', [['內容', orig, segment]])
        row += 1



### 5.3 用Python plotyly做出簡易儀錶板

In [40]:
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import collections
import pandas as pd

In [41]:
# 从Google Sheets中读取数据
gc = pygsheets.authorize(service_file='my-project-240608-425814-fb66d30e0809.json')
sh = gc.open('SEO分析結果')
wks = sh.worksheet_by_title('分詞結果')
data = wks.get_as_df()

# 准备数据用于可视化
data['Content_Word_Count'] = data['分詞結果'].apply(lambda x: len(x.split()))

# 创建函数来计算最常见的词语
def get_most_frequent_words(df, top_n=10):
    all_words = ' '.join(df['分詞結果']).split()
    word_counts = collections.Counter(all_words)
    most_common_words = word_counts.most_common(top_n)
    return pd.DataFrame(most_common_words, columns=['Word', 'Frequency'])

# 创建Dash应用
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("心理諮商相關結果的分詞數據分析"),
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': '標題', 'value': '標題'},
            {'label': 'Meta描述', 'value': 'Meta描述'},
            {'label': '內容', 'value': '內容'}
        ],
        value='標題'
    ),
    dcc.Graph(id='bar-chart')
])

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('dropdown', 'value')]
)
def update_graph(selected_category):
    filtered_df = data[data['類別'] == selected_category]
    most_frequent_words_df = get_most_frequent_words(filtered_df)
    fig = px.bar(most_frequent_words_df, x='Word', y='Frequency', title=f'{selected_category} 中最常出現的詞語')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)